In [1]:
import pandas as pd
import re
import ast

## RefCOCO

In [2]:
refcoco_results = pd.read_csv("/content/results_RefCOCO.csv")
refcoco_results.head(3)

,result,answer,code,id,query,img_path,possible_answers
0,"(149, 0, 433, 169)",[161.19999695 478.22000122 262.19000244 161.77...,"def execute_command_0(image, possible_answers,...",0,Please find the bounding box of the following ...,data/RefCOCO/images/refcoco-0.jpg,NaN
1,Bounding box of the second bag from the left: ...,[164.94999695 106.37000275 114.06999969 187.30...,"def execute_command_1(image, possible_answers,...",1,Please find the bounding box of the following ...,data/RefCOCO/images/refcoco-1.jpg,NaN
2,"(394, 166, 612, 425)",[408. 190.91000366 117.16000366 184.69...,"def execute_command_2(image, possible_answers,...",2,Please find the bounding box of the following ...,data/RefCOCO/images/refcoco-2.jpg,NaN


Extract the coordinates from predictions

In [3]:
def find_numbers(row):
    """Finds all complete numbers in the given string and returns them as a list."""
    return re.findall(r'\d+', str(row))

In [4]:
refcoco_results['result_processed'] = refcoco_results['result'].apply(find_numbers)

In [5]:
def find_abnormal(row):
  if len(row) == 4 or len(row) == 0:
    pass
  else:
    return row

Find all incorrect predictions

In [6]:
abnormal_result_values = refcoco_results[refcoco_results['result_processed'].apply(find_abnormal).notna()]
print(f"Number of abnormal results: {len(abnormal_result_values)}")

Number of abnormal results: 6


In [7]:
abnormal_result_values['result_processed']

,result_processed
39,"[2, 0, 0, 396, 256]"
252,"[2, 0, 150, 116, 420]"
423,"[2, 244, 24, 442, 340]"
537,"[2, 202, 91, 423, 312]"
685,"[3, 212, 49, 353, 298]"
774,"[2, 524, 39, 640, 305]"


Remove all the incorrect predictions

In [8]:
refcoco_results['result_processed'] = refcoco_results['result_processed'].apply(
    lambda x: [] if find_abnormal(x) is not None else x
)

Process the answers so they are in a list

In [9]:
def str_to_list(row):
  row = re.sub(r'\s+', ' ', row)
  row = row.replace('[ ', '[')
  row = row.replace(' ]', ']')
  string = row.replace(' ', ',')
  result = ast.literal_eval(string)
  return result

In [10]:
refcoco_results['answers_processed'] = refcoco_results['answer'].apply(str_to_list)
refcoco_results['answers_processed'].head(3)

,answers_processed
0,"[161.19999695, 478.22000122, 262.19000244, 161..."
1,"[164.94999695, 106.37000275, 114.06999969, 187..."
2,"[408.0, 190.91000366, 117.16000366, 184.69999695]"


In [11]:
to_analyze = refcoco_results[['answers_processed', 'result_processed']]
to_analyze.head(3)

,answers_processed,result_processed
0,"[161.19999695, 478.22000122, 262.19000244, 161...","[149, 0, 433, 169]"
1,"[164.94999695, 106.37000275, 114.06999969, 187...","[24, 167, 125, 261]"
2,"[408.0, 190.91000366, 117.16000366, 184.69999695]","[394, 166, 612, 425]"


In [12]:
print('Number of all samples:', len(to_analyze))
to_analyze = to_analyze[to_analyze['result_processed'].apply(lambda x: x != [])]
print('Number of all predicted samples:', len(to_analyze))

Number of all samples: 960
Number of all predicted samples: 713


Calculate the IoU metric

In [13]:
def iou(pred_box, true_box):
    """
    Calculate the Intersection over Union (IoU) between two bounding boxes.

    Parameters:
    - pred_box: List or tuple of (xmin, ymin, xmax, ymax) for the predicted bounding box
    - true_box: List or tuple of (xmin, ymin, xmax, ymax) for the ground truth bounding box

    Returns:
    - IoU value: A float between 0 and 1
    """

    # Coordinates of the predicted bounding box
    x1_pred, y1_pred, x2_pred, y2_pred = map(float, pred_box)
    # Coordinates of the ground truth bounding box
    x1_true, y1_true, x2_true, y2_true = map(float, true_box)

    # Calculate the coordinates of the intersection box
    x1_int = max(x1_pred, x1_true)
    y1_int = max(y1_pred, y1_true)
    x2_int = min(x2_pred, x2_true)
    y2_int = min(y2_pred, y2_true)

    # Calculate area of the intersection
    inter_area = max(0, x2_int - x1_int) * max(0, y2_int - y1_int)

    # Calculate area of both bounding boxes
    pred_area = (x2_pred - x1_pred) * (y2_pred - y1_pred)
    true_area = (x2_true - x1_true) * (y2_true - y1_true)

    # Calculate area of the union
    union_area = pred_area + true_area - inter_area

    # Calculate IoU
    iou_value = inter_area / union_area if union_area != 0 else 0

    return iou_value

In [14]:
to_analyze['IoU'] = to_analyze.apply(lambda row: iou(row['result_processed'], row['answers_processed']), axis=1)

In [15]:
to_analyze['IoU'].mean()

0.022783799000002693

## RefCOCO+

In [16]:
refcoco_plus_results = pd.read_csv("/content/results_RefCOCO+.csv")
refcoco_plus_results.head(3)

,result,answer,code,id,query,img_path,possible_answers
0,"(455, 357, 538, 428)",[468.29998779 0.91000003 171.69999695 116.12...,"import math\nfrom typing import List, Union\n\...",0,Please find the bounding box of the following ...,data/RefCOCO+/images/refcoco+-0.jpg,NaN
1,"(600, 262, 637, 307)",[289.67999268 110.25 290.76000977 365.35...,"def execute_command_1(image, possible_answers,...",1,Please find the bounding box of the following ...,data/RefCOCO+/images/refcoco+-1.jpg,NaN
2,"(71, 339, 98, 383)",[160. 246.33999634 110.88999939 240.78...,"def execute_command_2(image, possible_answers,...",2,Please find the bounding box of the following ...,data/RefCOCO+/images/refcoco+-2.jpg,NaN


In [17]:
refcoco_plus_results['result'].head()

,result
0,"(455, 357, 538, 428)"
1,"(600, 262, 637, 307)"
2,"(71, 339, 98, 383)"
3,"(79, 355, 196, 427)"
4,"(204, 241, 361, 499)"


In [18]:
refcoco_plus_results['result_processed'] = refcoco_plus_results['result'].apply(find_numbers)

In [19]:
abnormal_result_values = refcoco_plus_results[refcoco_plus_results['result_processed'].apply(find_abnormal).notna()]
print(f"Number of abnormal results: {len(abnormal_result_values)}")

Number of abnormal results: 7


In [20]:
abnormal_result_values['result_processed']

,result_processed
185,"[3, 532, 143, 614, 239]"
337,"[1, 5]"
459,"[1, 0, 313, 24, 348]"
787,"[4, 0, 0, 631, 344]"
797,"[10, 443, 343, 545, 437]"
897,"[579, 358, 640, 407, 450, 150, 525, 222]"
898,"[358, 196, 495, 284, 418, 215, 491, 283]"


In [21]:
refcoco_plus_results['result_processed'] = refcoco_plus_results['result_processed'].apply(
    lambda x: [] if find_abnormal(x) is not None else x
)

In [22]:
refcoco_plus_results['answers_processed'] = refcoco_plus_results['answer'].apply(str_to_list)
refcoco_plus_results['answers_processed'].head(3)

,answers_processed
0,"[468.29998779, 0.91000003, 171.69999695, 116.1..."
1,"[289.67999268, 110.25, 290.76000977, 365.3500061]"
2,"[160.0, 246.33999634, 110.88999939, 240.78999329]"


In [23]:
to_analyze_plus = refcoco_plus_results[['answers_processed', 'result_processed']]
to_analyze_plus.head(3)

,answers_processed,result_processed
0,"[468.29998779, 0.91000003, 171.69999695, 116.1...","[455, 357, 538, 428]"
1,"[289.67999268, 110.25, 290.76000977, 365.3500061]","[600, 262, 637, 307]"
2,"[160.0, 246.33999634, 110.88999939, 240.78999329]","[71, 339, 98, 383]"


In [24]:
print('Number of all samples:', len(to_analyze_plus))
to_analyze_plus = to_analyze_plus[to_analyze_plus['result_processed'].apply(lambda x: x != [])]
print('Number of all predicted samples:', len(to_analyze_plus))

Number of all samples: 1000
Number of all predicted samples: 903


In [25]:
to_analyze_plus['IoU'] = to_analyze_plus.apply(lambda row: iou(row['result_processed'], row['answers_processed']), axis=1)

In [26]:
to_analyze_plus['IoU'].mean()

0.03387825802874649